In [2]:
import torch
import numpy as np
import pandas as pd

In [5]:
! kaggle datasets download -d srinuti/residential-power-usage-3years-data-timeseries 


  0%|          | 0.00/227k [00:00<?, ?B/s]
100%|██████████| 227k/227k [00:00<00:00, 2.51MB/s]


In [9]:
df = pd.read_csv("data\power_usage_2016_to_2020.csv")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35952 entries, 0 to 35951
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StartDate    35952 non-null  object 
 1   Value (kWh)  35952 non-null  float64
 2   day_of_week  35952 non-null  int64  
 3   notes        35952 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [11]:
df.head()

,StartDate,Value (kWh),day_of_week,notes
0,2016-01-06 00:00:00,1.057,2,weekday
1,2016-01-06 01:00:00,1.171,2,weekday
2,2016-01-06 02:00:00,0.560,2,weekday
3,2016-01-06 03:00:00,0.828,2,weekday
4,2016-01-06 04:00:00,0.932,2,weekday
